In [94]:
import random

**Define empty board**

In [95]:
def gen_empty_board():
    return (('','',''),('','',''),('','',''))

In [96]:
S = gen_empty_board()

In [97]:
S

(('', '', ''), ('', '', ''), ('', '', ''))

In [98]:
def insert_a_sym(S,pos,sym):
    i,j = pos
    U = [list(t) for t in S]
    U[i][j] = sym
    return tuple(tuple(lst) for lst in U)

In [99]:
S = insert_a_sym(S,(1,1),'X')

In [100]:
S

(('', '', ''), ('', 'X', ''), ('', '', ''))

In [101]:
S[1][1]=='X'

True

**Define board values**

A board values table is a dictionary of values for board positions

In [102]:
board_values = {}

**Set initial board values**

<ul>
    <li>If there exist three 'X's are in a row, a column or a diagonal, the board value is set to $(1,0)$.</li>
    <li>If there exist three 'O's are in a row, a column or a diagonal, the board value is set to $(0,1)$.</li>
    <li>If there is no space in the board, the board value is set to $(0,0)$.</li>
    <li>For other board position, initial value is $(0.5,0.5)$.</li>
</ul>


Because we are not able to list out all the board positions, we use a function to generate board values.

In [103]:
def detect_three_in_a_row(S,sym):
    correct = False
    for i in range(3):
        if (S[i][0]==sym) and (S[i][1]==sym) and (S[i][2]==sym):
            correct = True
            break
        elif (S[0][i]==sym) and (S[1][i]==sym) and (S[2][i]==sym):
            correct = True
            break
    return correct

In [104]:
def detect_three_in_a_diagonal(S,sym):
    if (S[0][0]==sym) and (S[1][1]==sym) and (S[2][2]==sym):
        correct=True
    elif (S[2][0]==sym) and (S[1][1]==sym) and (S[0][2]==sym):
        correct=True
    else:
        correct=False
    return correct

In [105]:
def detect_empty_space(S):
    A = []
    for i in range(3):
        for j in range(3):
            if S[i][j]=='':
                A.append((i,j))
    if len(A)>0:
        return True
    else:
        return False

In [106]:
def gen_board_value(a_board):
    if len(board_values)==0:
        S=gen_empty_board()
        board_values[S]=(0.5,0.5)
    if a_board in board_values:
        return board_values[a_board]
    else:
        if detect_three_in_a_row(a_board,'X'):
            board_values[a_board]=(1,0)
        elif detect_three_in_a_diagonal(a_board,'X'):
            board_values[a_board]=(1,0)
        elif detect_three_in_a_row(a_board,'O'):
            board_values[a_board]=(0,1)
        elif detect_three_in_a_diagonal(a_board,'O'):
            board_values[a_board]=(0,1)
        elif detect_empty_space(a_board)==False:
            board_values[a_board] = (0,0)
        else:
            board_values[a_board]=(0.5,0.5)
        return board_values[a_board]

### Temporal difference learning

**To learn board values, we perform many random games.**


**If a board position $s$ leads (next move) to a board position $s^{'}$, we modify the board value for $s$ as $V(s) = V(s) + \alpha \big[V(s^{'})-V(s)\big]$, where $\alpha$ is a learning rate.**


**Assuming 'X' is the beginning player, we randomly insert 'X' and then 'O' to a board until one of the end game conditions is reached.**


**Assuming 'O' is the beginning player, we randomly insert 'O' and then 'X' to a board until one of the end game conditions is reached.**


**End game conditions**

<ul>
    <li>Three 'X' in a row, a column or a diagonal</li>
    <li>Three 'O' in a row, a column or a diagonal</li>
    <li>The board is full.</li>
</ul>

**This is a function to generate next possible board positions given the present board position and the next sym**

In [107]:
def next_possible_boards(U,sym):
    poss_boards = []
    for i in range(3):
        for j in range(3):
            V = U
            if V[i][j]=='':
                V = insert_a_sym(V,(i,j),sym)
                poss_boards.append(V)
    return poss_boards

In [108]:
S = gen_empty_board()

In [109]:
NS = next_possible_boards(S,'X')

In [110]:
random.choice(NS)

(('', '', ''), ('', '', ''), ('', '', 'X'))

**This is a function to play a random game**

In [111]:
def play_a_random_game(first_sym):
    S=[gen_empty_board()]
    if first_sym=='X':
        next_sym='O'
    else:
        next_sym='X'
    stop=0
    while stop==0:
        NS = next_possible_boards(S[-1],first_sym)
        if len(NS)>0:
            U = random.choice(NS)
            S.append(U)
            if detect_three_in_a_row(U,first_sym) or detect_three_in_a_diagonal(U,first_sym) or detect_empty_space(U)==False:
                stop=1
            else:
                NS = next_possible_boards(S[-1],next_sym)
                if len(NS)>0:
                    U = random.choice(NS)
                    S.append(U)
                    if detect_three_in_a_row(U,next_sym) or detect_three_in_a_diagonal(U,next_sym) or detect_empty_space(U)==False:
                        stop=1
                else:
                    stop=1
        else:
            stop=1
    return S

In [112]:
play_a_random_game('X')

[(('', '', ''), ('', '', ''), ('', '', '')),
 (('', 'X', ''), ('', '', ''), ('', '', '')),
 (('', 'X', ''), ('', '', ''), ('O', '', '')),
 (('', 'X', ''), ('', 'X', ''), ('O', '', '')),
 (('', 'X', ''), ('O', 'X', ''), ('O', '', '')),
 (('', 'X', ''), ('O', 'X', 'X'), ('O', '', '')),
 (('', 'X', ''), ('O', 'X', 'X'), ('O', 'O', '')),
 (('', 'X', 'X'), ('O', 'X', 'X'), ('O', 'O', '')),
 (('O', 'X', 'X'), ('O', 'X', 'X'), ('O', 'O', ''))]

**A function to modify board values**

In [113]:
def modify_board_values(a_game,alpha):
    for i in range(1,len(a_game)+1):
        if i==1:
            board_values[a_game[-i]]=gen_board_value(a_game[-i])
        else:
            board_values[a_game[-i]] = gen_board_value(a_game[-i])
            if board_values[a_game[-i]] != (0,0) or board_values[a_game[-i]] != (1,0) or board_values[a_game[-i]] != (0,1):
                p = max(0,min(1,board_values[a_game[-i]][0] + alpha*(board_values[a_game[-i+1]][0] 
                                                                     - board_values[a_game[-i]][0])))
                board_values[a_game[-i]] = (p,1-p)

In [114]:
for _ in range(5000):
    modify_board_values(play_a_random_game('X'),0.1)
    modify_board_values(play_a_random_game('O'),0.1)

In [115]:
for i, (key, value) in enumerate(board_values.items()):
    if i < 10:
        print(f'{key}: {value}')
    else:
        break

(('', '', ''), ('', '', ''), ('', '', '')): (0.4981371423197026, 0.5018628576802975)
(('X', 'O', 'O'), ('O', 'X', 'X'), ('X', 'X', 'O')): (0, 0)
(('', 'O', 'O'), ('O', 'X', 'X'), ('X', 'X', 'O')): (0.06078832729528464, 0.9392116727047154)
(('', 'O', ''), ('O', 'X', 'X'), ('X', 'X', 'O')): (0.5053922857945, 0.4946077142055)
(('', 'O', ''), ('O', 'X', 'X'), ('X', '', 'O')): (0.549266320860024, 0.450733679139976)
(('', '', ''), ('O', 'X', 'X'), ('X', '', 'O')): (0.5063075541126267, 0.49369244588737327)
(('', '', ''), ('O', 'X', 'X'), ('', '', 'O')): (0.49923992916266, 0.50076007083734)
(('', '', ''), ('O', 'X', 'X'), ('', '', '')): (0.4970022265763433, 0.5029977734236567)
(('', '', ''), ('O', 'X', ''), ('', '', '')): (0.514107171640257, 0.485892828359743)
(('', '', ''), ('', 'X', ''), ('', '', '')): (0.5122385621167046, 0.4877614378832954)


**A function to find next best move given the present board position and next symbol**

In [116]:
def find_next_best_move(S,sym):
    if sym=='X':
        if gen_board_value(S)==(1,0):
            return 'Already win for X'
        elif gen_board_value(S)==(0,1):
            return 'Already lost for X'
        elif gen_board_value(S)==(0,0):
            return 'Draw'
        else:
            NS = next_possible_boards(S,sym)
            A = []
            for x in NS:
                if x in board_values:
                    A.append(board_values[x][0])
                else:
                    A.append(gen_board_value(x)[0])
            u = max(A)
            return random.choice([x for x in NS if board_values[x][0]==u])
    else:
        if gen_board_value(S)==(0,1):
            return 'Already win for O'
        elif gen_board_value(S)==(1,0):
            return 'Already lost for O'
        elif gen_board_value(S)==(0,0):
            return 'Draw'
        else:
            NS = next_possible_boards(S,sym)
            A = []
            for x in NS:
                if x in board_values:
                    A.append(board_values[x][1])
                else:
                    A.append(gen_board_value(x)[1])
            u = max(A)
            return random.choice([x for x in NS if board_values[x][1]==u])

In [117]:
S = (('X', '', 'X'), ('O', 'X', 'X'), ('O', '', 'O'))
find_next_best_move(S,'X')

(('X', 'X', 'X'), ('O', 'X', 'X'), ('O', '', 'O'))

In [118]:
S = (('X', '', ''), ('O', '', ''), ('X', '', ''))
find_next_best_move(S,'O')

(('X', '', ''), ('O', 'O', ''), ('X', '', ''))

**Generate an optimal game for 'X' when the given sym player begins first**

In [119]:
def gen_an_optimal_game(start_sym):
    if start_sym=='X':
        next_sym='O'
    else:
        next_sym='X'
    S = [gen_empty_board()]
    stop = 0
    while stop==0:
        S.append(find_next_best_move(S[-1],start_sym))
        if (S[-1] != f'Already win for {start_sym}') and (S[-1] != f'Already lost for {start_sym}') and (S[-1] != 'Draw'):
            S.append(find_next_best_move(S[-1],next_sym))
            if (S[-1]==f'Already win for {next_sym}') or (S[-1]==f'Already lost for {next_sym}') or (S[-1]=='Draw'):
                stop=1
        else:
            stop=1
    return S

In [120]:
gen_an_optimal_game('X')

[(('', '', ''), ('', '', ''), ('', '', '')),
 (('', '', ''), ('', 'X', ''), ('', '', '')),
 (('', '', ''), ('', 'X', ''), ('', '', 'O')),
 (('', '', ''), ('', 'X', ''), ('', 'X', 'O')),
 (('', '', ''), ('', 'X', 'O'), ('', 'X', 'O')),
 (('', 'X', ''), ('', 'X', 'O'), ('', 'X', 'O')),
 'Already lost for O']

In [121]:
gen_an_optimal_game('O')

[(('', '', ''), ('', '', ''), ('', '', '')),
 (('', '', ''), ('', 'O', ''), ('', '', '')),
 (('', '', 'X'), ('', 'O', ''), ('', '', '')),
 (('O', '', 'X'), ('', 'O', ''), ('', '', '')),
 (('O', '', 'X'), ('', 'O', 'X'), ('', '', '')),
 (('O', '', 'X'), ('', 'O', 'X'), ('', '', 'O')),
 'Already lost for X']